In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import re as re

import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit


# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.callbacks import History,LearningRateScheduler
# from tensorflow.keras.layers import Dropout
# print('TensorFlow %s, Keras %s, numpy %s, pandas %s'%(tf.__version__,keras.__version__, np.__version__,pd.__version__))
__DEBUG__=False

In [9]:
train = pd.read_csv('data_concatener.csv', sep=';')
test  = pd.read_csv('test_set_values.csv')
full_data = [train, test]
finalfile_index=test.id #Index des données de test pour le résultat final

# Useless column
train.drop('num_private', axis=1, inplace=True)
train.drop('date_recorded', axis=1, inplace=True)
train.drop('funder', axis=1, inplace=True)
train.drop('installer', axis=1, inplace=True)
train.drop('scheme_name', axis=1, inplace=True)
train.drop('scheme_management', axis=1, inplace=True)
train.drop('recorded_by', axis=1, inplace=True)
train.drop('latitude', axis=1, inplace=True)
train.drop('longitude', axis=1, inplace=True)
test.drop('num_private', axis=1, inplace=True)
test.drop('date_recorded', axis=1, inplace=True)
test.drop('funder', axis=1, inplace=True)
test.drop('installer', axis=1, inplace=True)
test.drop('scheme_name', axis=1, inplace=True)
test.drop('scheme_management', axis=1, inplace=True)
test.drop('recorded_by', axis=1, inplace=True)
test.drop('latitude', axis=1, inplace=True)
test.drop('longitude', axis=1, inplace=True)

# Double column
train.drop('quantity', axis=1, inplace=True)
train.drop('source', axis=1, inplace=True)
train.drop('waterpoint_type', axis=1, inplace=True)
train.drop('payment', axis=1, inplace=True)
train.drop('management', axis=1, inplace=True)
train.drop('extraction_type', axis=1, inplace=True)
test.drop('quantity', axis=1, inplace=True)
test.drop('source', axis=1, inplace=True)
test.drop('waterpoint_type', axis=1, inplace=True)
test.drop('payment', axis=1, inplace=True)
test.drop('management', axis=1, inplace=True)
test.drop('extraction_type', axis=1, inplace=True)

# Refactoring value empty
for dataset in full_data:
    # dataset.loc[dataset.latitude == '-2,00E-08', 'latitude'] = float('0.0')
    # dataset.loc[dataset.latitude.replace('.',',')]
    # dataset.loc['latitude'] = dataset['latitude'].astype(float)
    dataset.loc[dataset.public_meeting.isnull(), 'public_meeting'] = 0
    dataset.loc[dataset.public_meeting == False, 'public_meeting'] = 0
    dataset.loc[dataset.public_meeting == True, 'public_meeting'] = 1
    # dataset.loc[dataset.funder.isnull(), 'funder'] = 'Unknown'
    # dataset.loc[dataset.installer.isnull(), 'installer'] = 'Unknown'
    dataset.loc[dataset.subvillage.isnull(), 'subvillage'] = 'None'
    # dataset.loc[dataset.scheme_management.isnull(), 'scheme_management'] = 'Other'
    # dataset.loc[dataset.scheme_name.isnull(), 'scheme_name'] = 'Unknown'
    dataset.loc[dataset.permit.isnull(), 'permit'] = 0
    dataset.loc[dataset.permit == False, 'permit'] = 0
    dataset.loc[dataset.permit == True, 'permit'] = 1
    dataset.loc[dataset.construction_year == 0, 'construction_year'] = np.NaN
    construction_year_avg = dataset['construction_year'].mean() # Calcul de la valeur moyenne
    construction_year_std = dataset['construction_year'].std()  # Calcul de l'écart type
    construction_year_null_count = dataset['construction_year'].isnull().sum() # nombre de valeur nulle
    construction_year_null_random_list = np.random.randint(construction_year_avg - construction_year_std, construction_year_avg + construction_year_std, size=construction_year_null_count)   
    dataset.loc[np.isnan(dataset['construction_year']),'construction_year'] = construction_year_null_random_list    
    dataset['construction_year'] = dataset['construction_year'].astype(int)

#La fonction info() permet de répérer les colonnes avec des valeurs nulles
print('\nTrain data:')
train.info()
print('\nTest data:')
test.info()
# print(train['public_meeting'])
# Final output of the training data
train.to_excel('output_final.xlsx')


Train data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 26 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
gps_height               59400 non-null int64
wpt_name                 59400 non-null object
basin                    59400 non-null object
subvillage               59400 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           59400 non-null int64
permit                   59400 non-null int64
construction_year        59400 non-null int32
extraction_type_group    59400 non-null object
extraction_type_class    59400 non-null object
management_group         59400 non-null object
payment_type             59400 non-null